In [1]:
import pandas as pd
import os
import sys


def completeness_template(session):
    
    # check a "examplar session" to extract a list of acqusitions, acqusition length, and phase encoding direction
    
    df = {"Session": [],
          "Modality": [],
          "SeriesNumber": [],
          "SeriesDescription": [],
          "AcquisitionLabel": [],
          "ExpectedPEDir": [],
          "ExpectedLength": []}
    
    for acq in session.acquisitions():
        dcm_file = get_dicom_file(acq)
        if "SeriesDescription" not in dcm_file.info:
            continue
            
        df["Session"] = session.label
        df["Modality"].append(acq.label.split("-")[0] if acq.label.split("-")[0] in ['anat','func','fmap','dwi'] else "scout")
        df["SeriesNumber"].append(dcm_file.info["SeriesNumber"])
        df["SeriesDescription"].append(dcm_file.info["SeriesDescription"])
        df["AcquisitionLabel"].append(acq.label)
        
        if any(f in ["BOLD","dti","B0"] for f in dcm_file.classification["Measurement"] ):
            nifti_file = get_nifti_file(acq)
            df["ExpectedLength"].append(dcm_file.zip_member_count)
            df["ExpectedPEDir"].append(nifti_file.info["PhaseEncodingDirection"])
        else:
            df["ExpectedLength"].append(None)
            df["ExpectedPEDir"].append(None)
    return df
        

def get_dicom_file(acq):
    acq = fw.get(acq.id)
    return next((f for f in acq.files if f.type == "dicom"), None)

def get_nifti_file(acq):
    acq = fw.get(acq.id)
    return next((f for f in acq.files if f.type == "nifti"), None)
    



In [2]:
import flywheel
fw = flywheel.Client('')
session = fw.get(fw.lookup('<path-to-session>').id)

json = completeness_template(session)
df = pd.DataFrame.from_dict(json)
df = df.sort_values("SeriesNumber", ignore_index=True)
df = df.drop('SeriesNumber', axis=1)

# write out as output
project = fw.get(session.parents["project"]).label.lower()
df.to_csv(f'{project}_completeness.csv', index=False)
df